# 🤖 Multilayer Perceptron

Esta Jupyter Notebook implementa una MLP para detectar patrones en una matriz 10x10.
Se genera un dataset para luego entrenar el modelo y validarlo.
Se evalúa la precisión para reiterar sobre el modelo con el objetivo de mejorar su eficacia.
Finalmente se exporta el modelo para ser utilizado en la aplicación web.


## 📋 Situación

En una matriz 10x10 se debe poder detectar las letras `b`, `d`, `f` que se corresponden a los siguientes patrones:

![Patrones que el MLP deberá reconocer](assets/patterns.png)

Ante un dato nuevo, el MLP deberá ser capaz de clasificar el contenido de esa matriz en uno de los tres patrones.


In [1]:
import pandas as pd
import numpy as np
from typing import Literal, List, Dict

# Extend maximum width when printing DataFrames so they fit in just one line
pd.options.display.width = 100

# Set seed for reproducibility
SEED = 7
RNG = np.random.default_rng(SEED)

# fmt: off
# Define the 10x10 patterns for 'b', 'd', and 'f' as a 1D array
PATTERNS = {
    "b": np.array(
           [
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
            0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
            0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
            0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
            0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        ],
        dtype=np.uint8,
    ),
    "d": np.array(
        [
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
            0, 0, 0, 1, 1, 1, 1, 1, 0, 0,
            0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
            0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
            0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
            0, 0, 0, 1, 1, 1, 1, 1, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        ],
        dtype=np.uint8,
    ),
    "f": np.array(
        [
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
            0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
            0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
            0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
            0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        ],
        dtype=np.uint8,
    ),
}
# fmt: on

## 🗃️ Generación de Datasets

Conviene generar el conjunto de datos de manera programática.
Los datasets deberán ser representativos a la hora de definir la distribución de los ejemplos de entrenamiento.
Se definen los patrones 'b', 'd', y 'f'.
Luego, se crea una función `generate_sample` que crea un ejemplo de un patrón dado con una distorsión entre 0 y 0.3. 


In [2]:
def generate_sample(pattern: Literal["b", "d", "f"], noise: float = 0.0) -> np.ndarray:
    """
    Generates a 1D array based on a given pattern letter ('b', 'd', or 'f'), with optional noise.

    Args:
        pattern: One of 'b', 'd', or 'f'.
        noise: Proportion of pixels to flip.
    Returns:
        A numpy 1D array representing the 10x10 matrix with the pattern and noise applied.
    """
    sample = PATTERNS[pattern].copy()
    num_pixels = sample.size
    num_noisy = int(noise * num_pixels)

    if num_noisy > 0:
        # Choose random indices to flip
        indices = RNG.choice(num_pixels, num_noisy, replace=False)
        # Flip the selected pixels (0 becomes 1, 1 becomes 0)
        sample[indices] = 1 - sample[indices]

    return sample


def print_sample(sample: np.ndarray):
    """
    Pretty-prints the provided 1D array as a 10x10 matrix,
    coloring filled pixels bold green and adding an outline to the matrix.

    Args:
        sample: 1D numpy array of 0s and 1s.
    """
    BOLD_GREEN = "\033[1;92m"
    RESET = "\033[0m"
    for i in range(10):
        row = sample[i * 10 : (i + 1) * 10]
        print(" ".join([f"{BOLD_GREEN}1{RESET}" if val else "0" for val in row]))


print("A 'b' pattern with zero noise:")
print_sample(generate_sample("d", 0.00))
print()
print("A 'b' with 30% noise, meaning 30 out of 100 cells have been randomly flipped:")
print_sample(generate_sample("d", 0.30))

A 'b' pattern with zero noise:
0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 1 0 0
0 0 0 0 0 0 0 1 0 0
0 0 0 0 0 0 0 1 0 0
0 0 0 1 1 1 1 1 0 0
0 0 1 0 0 0 0 1 0 0
0 0 1 0 0 0 0 1 0 0
0 0 1 0 0 0 0 1 0 0
0 0 0 1 1 1 1 1 0 0
0 0 0 0 0 0 0 0 0 0

A 'b' with 30% noise, meaning 30 out of 100 cells have been randomly flipped:
1 0 0 0 1 0 0 0 0 0
1 1 0 0 0 0 0 0 0 0
0 0 0 1 1 0 1 0 0 0
0 1 0 0 0 0 0 1 0 0
0 0 1 0 1 0 1 0 0 1
1 0 1 0 0 0 0 1 1 0
0 0 1 0 1 0 1 0 1 0
1 1 1 0 1 1 0 1 0 0
0 0 0 1 1 1 1 0 0 1
0 0 0 0 0 1 1 1 0 0


Luego se generan 3 datasets que contengan 100, 500 y 1000 ejemplos. 
El 10% serán patrones sin distorsionar y el resto con una distorsión del 1% al 30%.
Se usará una **distribución uniforme** para ubicar el 90% de ejemplos en el rango de distorsión entre 0.01 y 0.30.

Cada dataset será un `pd.DataFrame` de `pandas` que contiene columnas del `0` al `99` (una por cada celda de la matriz) y una columna final `class` que indica la clase del patrón ('b', 'd' o 'f').


In [3]:
def generate_dataset(n_samples: int) -> pd.DataFrame:
    """
    Generates a dataset of pattern samples.
    The 90% of samples will have noise between 0.01 and 0.30.

    Args:
        n_samples: Number of samples to generate.
    Returns:
        A dataframe with 100 columns for the flattened pattern and 1 column 'class' for the pattern class.
    """
    columns = [str(i) for i in range(100)] + ["class"]
    df = pd.DataFrame(0, index=np.arange(n_samples), columns=columns)
    df = df.astype({"class": "str"})

    for i in range(n_samples):
        # 10% without distortion, 90% with distortion between 1% and 30%
        if i < int(0.1 * n_samples):
            noise = 0.0
        else:
            noise = RNG.uniform(0.01, 0.30)

        # Pick a pattern at random
        pattern = RNG.choice(list(PATTERNS.keys()))

        sample = generate_sample(pattern, noise).flatten()
        df.iloc[i, :100] = sample
        df.loc[i, "class"] = pattern

    return df


df_100 = generate_dataset(100)
df_500 = generate_dataset(500)
df_1000 = generate_dataset(1000)

print("Dataset de 1000 ejemplos:")
print(df_1000)

Dataset de 1000 ejemplos:
     0  1  2  3  4  5  6  7  8  9  ...  91  92  93  94  95  96  97  98  99  class
0    0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      d
1    0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      f
2    0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      f
3    0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      b
4    0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      b
..  .. .. .. .. .. .. .. .. .. ..  ...  ..  ..  ..  ..  ..  ..  ..  ..  ..    ...
995  0  0  0  1  0  1  0  0  0  0  ...   0   0   0   0   0   1   0   0   0      b
996  0  0  0  1  0  0  0  0  0  0  ...   0   0   0   0   0   0   1   0   1      f
997  0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      b
998  0  1  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      b
999  1  0  0  0  0  1  1  0  0  1  ...   0   1   0   0   0   1   1   1  

Por cada dataset deberán construirse tres conjuntos de validación con 10%, 20% y 30% de los ejemplos.
Esto da un total de 9 pares de datasets de entrenamiento y datasets de validación:

| Dataset | Cantidad de Ejemplos | % Entrenamiento | % Validación |
|----|--------------------|-----------------|-------------|
|  1 | 100                | 90%             | 10%         |
|  2 | 100                | 80%             | 20%         |
|  3 | 100                | 70%             | 30%         |
|  4 | 500                | 90%             | 10%         |
|  5 | 500                | 80%             | 20%         |
|  6 | 500                | 70%             | 30%         |
|  7 | 1000               | 90%             | 10%         |
|  8 | 1000               | 80%             | 20%         |
|  9 | 1000               | 70%             | 30%         |

In [4]:
VALIDATION_RATIOS = [0.1, 0.2, 0.3]


def split_dataset(df: pd.DataFrame, validation_ratio: float):
    """
    Splits the dataset into training and validation sets.

    Args:
        df: The dataset to split.
        validation_ratio: Proportion (between 0 and 1) of the dataset to include in the validation set.
    Returns:
        tuple: (training_df, validation_df).
    """
    shuffled_df = df.sample(frac=1, random_state=SEED).reset_index(drop=True)
    validation_size = int(len(df) * validation_ratio)
    validation_df = shuffled_df.iloc[:validation_size].reset_index(drop=True)
    training_df = shuffled_df.iloc[validation_size:].reset_index(drop=True)
    return training_df, validation_df


# Store all datasets in one dictionary
datasets = {}
for df_name, df in [("100", df_100), ("500", df_500), ("1000", df_1000)]:
    datasets[df_name] = {}
    for validation_ratio in VALIDATION_RATIOS:
        training, validation = split_dataset(df, validation_ratio)
        key = f"val_{int(validation_ratio * 100)}%"
        # Store training dataset and validation dataset for each dataset, for each validation split
        datasets[df_name][key] = {
            "training": training,
            "validation": validation,
        }

## 🧠 Implementación del Modelo

Se debe implementar el algoritmo MLP que permita, dado un dataset, parametrizar la cantidad de capas, neuronas y funciones de activación con los que se entrenará la red neuronal.

Requerimientos para la arquitectura del modelo:

- 1 o 2 capas ocultas.
- De 5 a 10 neuronas por capa.
- Funciones de activación: lineal y sigmoidal.
- Coeficiente de aprendizaje entre 0 y 1.
- Término momento entre 0 y 1.

Para estudiar el algoritmo se utiliza de referencia la bibliografía recomendada por la cátedra [disponible en el campus virtual](https://frre.cvg.utn.edu.ar/pluginfile.php/105673/mod_label/intro/Perceprtr%C3%B3n-MLP.pdf):

- Hilera, J. R. Martinez, V. J. (2000) Redes Neuronales Artificiales. Fundamentos, modelos y aplicaciones. Alfaomega.

También se consulta material complementario para resolver dudas durante la implementación:

- Prince, S. J. D. (2023). Understanding deep learning. The MIT Press. [http://udlbook.com](http://udlbook.com).

### Arquitectura

La arquitectura de esta red neuronal feedforward (MLP) consiste en:

- **Capa de entrada:** 100 neuronas, una por cada celda de la matriz.
- **Capa oculta 1:** 10 neuronas.
- **Capa oculta 2:** 5 neuronas.
- **Capa de salida:** 3 neuronas, una para la clasificación de cada patrón 'b', 'd', 'f'.

Próximamente los hiperparámetros serán parametrizables, pero se utiliza esta arquitectura como base.

![Diagrama de la arquitectura del MLP](assets/architecture.png)

Esta red neuronal será una función $f[X, \phi] = Y$ que clasifica el contenido de una matriz 10x10 en uno de tres patrones. 
El contenido de la matriz 10x10 se representa como el vector de entrada $X = [x_0, x_1, \dots, x_{99}]^T$.
El vector de salida $Y = [y_b, y_d, y_f]^T$ contiene la predicción del modelo para cada patrón.
El conjunto de parámetros $\phi = \set{B_k, W_k}_{k=0}^K$ del modelo contiene:

- El vector de _biases_ $B_k$ que contribuyen a la capa $k+1$. Es de tamaño $D_{k+1}$.
- Los _weights_ (pesos) $W_k$ que son aplicados a la capa $k$ y que contribuyen a la capa $k+1$. Es de tamaño $D_{k+1}  \times D_k$.

La red neuronal se puede representar utilizando notación matricial:

$$
\begin{aligned}
X &= [x_1, x_2, \dots, x_{100}]^T \\
H_1 &= a \left[ B_{1} + W_{1} X \right] \\
H_2 &= a \left[ B_{2} + W_{2} H_1 \right] \\
Y &= B_3 + W_{3} H_2 \\
\end{aligned}
$$

donde $a[\bullet]$ es una función que aplica la función de activación por separado a cada elemento de su vector de entrada (Prince, 2023).

Esa notación se puede expandir de la siguiente manera:

$$\begin{aligned}
H_1
&=
\begin{bmatrix}
h_1 \\
h_2 \\
\vdots \\
h_{10}
\end{bmatrix}
= 
a
\left[
\begin{bmatrix}
b_{1} \\
b_{2} \\
\vdots \\
b_{10}
\end{bmatrix}
+
\begin{bmatrix}
w_{11} & w_{12} & \dots & w_{1\ 100} \\
w_{21} & w_{22} & \dots & w_{2\ 100} \\
\vdots & \vdots & \ddots & \vdots \\
w_{10 \ 1} & w_{10\ 2} & \dots & w_{10\ 100} \\
\end{bmatrix}
\begin{bmatrix}
x_{1} \\
x_{2} \\
\vdots \\
x_{100}
\end{bmatrix}
\right]  \\

H_2
&=
\begin{bmatrix}
h'_1 \\
h'_2 \\
\vdots \\
h'_5
\end{bmatrix}
= 
a
\left[
\begin{bmatrix}
b'_{1} \\
b'_{2} \\
\vdots \\
b'_{5}
\end{bmatrix}
+
\begin{bmatrix}
w'_{11} & w'_{12} & \dots & w'_{1\ 10} \\
w'_{21} & w'_{22} & \dots & w'_{2\ 10} \\
\vdots & \vdots & \ddots & \vdots \\
w'_{51} & w'_{52} & \dots & w'_{5\ 10} \\
\end{bmatrix}
\begin{bmatrix}
h_{1} \\
h_{2} \\
\vdots \\
h_{10}
\end{bmatrix}
\right]  \\

Y
&=
\begin{bmatrix}
y_b \\
y_d \\
y_f
\end{bmatrix}
= 
\begin{bmatrix}
b''_{1} \\
b''_{2} \\
b''_{3}
\end{bmatrix}
+
\begin{bmatrix}
w''_{11} & w''_{12} & \dots & w''_{14} \\
w''_{21} & w''_{22} & \dots & w''_{24} \\
w''_{31} & w''_{32} & \dots & w''_{34} \\
\end{bmatrix}
\begin{bmatrix}
h'_{1} \\
h'_{2} \\
\vdots \\
h'_{5}
\end{bmatrix} \\
\end{aligned}
$$

Por ejemplo, la segunda neurona de la primera capa oculta se calcula como $h_1 = a[b_1 + w_{21}x_0 + \dots + w_{2\ 100}x_{100}]$.

Los parámetros de $\phi$ serán inicializados utilizando la técnica de Xavier Glorot, y el entrenamiento del MLP consistirá en ajustar estos parámetros hasta lograr resultados aceptables.

### Funciones de Activación

Si bien existen múltiples funciones de activación, el trabajo práctico requiere dos en particular:

- Lineal.
- Sigmoidal.

![Funciones de activación](assets/activation_functions.png)


### Regla de Aprendizaje

Se debe elegir la regla de aprendizaje o _loss function_ a utilizar.
Hilera & Martinez presentan la regla _Least Mean Squared_ o _regla delta_:

$$
\epsilon_k^2 = \frac{1}{2L} \sum_{k=1}^L (d_k - s_k)^2
$$


mientras que Prince presenta una función _Least Squares_ muy similar que omite el coeficiente constante:

$$
L[\phi] = \sum_{i=1}^I (y_i - f[X_i, \phi])^2
$$

Ambas consisten en comparar la salida obtenida contra la deseada para obtener el costo o pérdida.
El costo se debería reducir en cada iteración del entrenamiento del MLP a medida que se ajustan los parámetros.


### Momento

El _momento_ es una modificación al algoritmo _backpropagation_ del gradiente descendiente para suavizar el progreso del algoritmo y evitar oscilaciones.
Matemáticamente es un coeficiente $\beta$ que se agrega para considerar el valor de la iteración anterior de un parámetro al momento de calcular su nuevo valor en la iteración siguiente.

Hilera & Martinez lo presentan como:

$$
\begin{aligned}
w_{ji} (t+1) &= w_{ji}(t) + \alpha \ \delta_{pj}y_{pi} + \underbrace {\beta (w_{ji}(t) - w_{ji} (t-1))}_\text{Término momento} \\
\Delta w_{ji} (t+1) &= \alpha \ \delta_{pj}y_{pi} + \beta \ \Delta w_{ji} (t) \\
\end{aligned}
$$

mientras que Prince lo presenta como:

$$
\begin{aligned}
\mathbf{m}_{t+1} &\leftarrow \beta \cdot \mathbf{m}_t 
  + (1 - \beta) \sum_{i \in \mathcal{B}_t} 
  \frac{\partial \ell_i[\boldsymbol{\phi}_t]}{\partial \boldsymbol{\phi}} \\[6pt]
\boldsymbol{\phi}_{t+1} &\leftarrow 
  \boldsymbol{\phi}_t - \alpha \cdot \mathbf{m}_{t+1}
\end{aligned}
$$

In [ ]:
def initialize_parameters() -> Dict[str, np.ndarray]:
    """
    Initialize weights and biases using Xavier initialization, where each
    parameter will be drawn from a normal distribution with mean of `0` and
    a standard deviation of `sqrt(2 / (input_layer_size + output_layer_size))`.
    This helps prevent vanishing or exploding gradients.
    """
    # Initialize layer 1 parameters (input -> hidden1)
    std1 = np.sqrt(2.0 / (100 + 10))
    W1 = RNG.normal(0, std1, (10, 100))
    b1 = np.zeros(10)

    # Initialize layer 2 parameters (hidden1 -> hidden2)
    std2 = np.sqrt(2.0 / (10 + 5))
    W2 = RNG.normal(0, std2, (5, 10))
    b2 = np.zeros(5)

    # Initialize output layer parameters (hidden2 -> output)
    std3 = np.sqrt(2.0 / (5 + 3))
    W3 = RNG.normal(0, std3, (3, 5))
    b3 = np.zeros(3)

    # Initialize momentum terms
    dW1_prev = np.zeros_like(W1)
    db1_prev = np.zeros_like(b1)
    dW2_prev = np.zeros_like(W2)
    db2_prev = np.zeros_like(b2)
    dW3_prev = np.zeros_like(W3)
    db3_prev = np.zeros_like(b3)

    print("Parameters initialized:")
    print(f"  W1 shape: {W1.shape}, b1 shape: {b1.shape}")
    print(f"  W2 shape: {W2.shape}, b2 shape: {b2.shape}")
    print(f"  W3 shape: {W3.shape}, b3 shape: {b3.shape}")
    print(
        f"  Total parameters: {W1.size + b1.size + W2.size + b2.size + W3.size + b3.size}"
    )

    return {
        "W1": W1,
        "b1": b1,
        "W2": W2,
        "b2": b2,
        "W3": W3,
        "b3": b3,
        "dW1_prev": dW1_prev,
        "db1_prev": db1_prev,
        "dW2_prev": dW2_prev,
        "db2_prev": db2_prev,
        "dW3_prev": dW3_prev,
        "db3_prev": db3_prev,
    }


def get_activation_function(activation_type: Literal["sigmoid", "linear"] = "sigmoid"):
    """Return the activation function based on the activation type."""
    if activation_type == "sigmoid":
        return lambda x: 1 / (1 + np.exp(-x))
    elif activation_type == "linear":
        return lambda x: x


def get_activation_derivative(
    activation_type: Literal["sigmoid", "linear"] = "sigmoid",
):
    """Return the derivative of the activation function based on the activation type."""
    if activation_type == "sigmoid":
        return lambda x: 1 / (1 + np.exp(-x)) * (1 - 1 / (1 + np.exp(-x)))
    elif activation_type == "linear":
        return lambda x: np.ones_like(x)


def feedforward(
    X: np.ndarray,
    params: dict,
    activation_type: Literal["sigmoid", "linear"] = "sigmoid",
) -> np.ndarray:
    """
    Perform a feedforward pass through the network.

    Args:
        X: Input pattern (100 values).
        params: Dictionary containing weights and biases.
        activation_type: Name of activation function to use ("sigmoid" or "linear").
    Returns:
        Array with all neuron activations at each layer [X, h1, h2, y].
    """
    a = get_activation_function(activation_type)

    # Forward pass from input layer to hidden layer 1
    h1 = a(params["b1"] + np.dot(params["W1"], X.T).T)

    # Forward pass from hidden layer 1 to hidden layer 2
    h2 = a(params["b2"] + np.dot(params["W2"], h1.T).T)

    # Forward pass from hidden layer 2 to output layer
    y = params["b3"] + np.dot(params["W3"], h2.T).T

    return [X, h1, h2, y]


def calculate_loss(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """
    Compute the mean squared error loss.

    Args:
        y_true: Ground truth target values.
        y_pred: Predicted values from the model.
    Returns:
        Mean squared error between y_true and y_pred.
    """
    return np.mean((y_true - y_pred) ** 2)


def train_step(
    X: np.ndarray,
    y_target: np.ndarray,
    params: Dict[str, np.ndarray],
    learning_rate: float = 0.1,
    momentum: float = 0.1,
    activation_type: Literal["sigmoid", "linear"] = "sigmoid",
) -> Dict[str, np.ndarray]:
    """
    Perform one training step: forward pass, loss calculation and backpropagation.

    Args:
        X: Input pattern (100 values).
        y_target: Expected output (array with 3 values, 1 for each class).
        params: Dictionary containing weights and biases.
        learning_rate: Learning rate for parameter updates.
        momentum: Momentum coefficient for parameter updates.
        activation_type: Type of activation function.
    Returns:
        Dictionary containing updated weights and biases.
    """
    _, h1, h2, y = feedforward(X, params, activation_type)

    # Get activation derivative function
    a_derived = get_activation_derivative(activation_type)

    # Backward pass
    dL_dz3 = 2 * (y - y_target)  # Derivative of loss function
    dL_dW3 = np.outer(dL_dz3, h2)  # Gradient w.r.t. W3 (3,5)
    dL_db3 = dL_dz3  # Gradient w.r.t. b3

    dL_dh2 = np.dot(dL_dz3, params["W3"])  # Gradient w.r.t. h2
    dL_dz2 = dL_dh2 * a_derived(h2)  # Gradient w.r.t. pre-activation of h2
    dL_dW2 = np.outer(dL_dz2, h1)  # Gradient w.r.t. W2 (5,10)
    dL_db2 = dL_dz2  # Gradient w.r.t. b2

    dL_dh1 = np.dot(dL_dz2, params["W2"])  # Gradient w.r.t. h1
    dL_dz1 = dL_dh1 * a_derived(h1)  # Gradient w.r.t. pre-activation of h1
    dL_dW1 = np.outer(dL_dz1, X)  # Gradient w.r.t. W1 (10,100)
    dL_db1 = dL_dz1  # Gradient w.r.t. b1

    # Update parameters with momentum
    dW1_update = -learning_rate * dL_dW1 + momentum * params["dW1_prev"]
    db1_update = -learning_rate * dL_db1 + momentum * params["db1_prev"]

    dW2_update = -learning_rate * dL_dW2 + momentum * params["dW2_prev"]
    db2_update = -learning_rate * dL_db2 + momentum * params["db2_prev"]

    dW3_update = -learning_rate * dL_dW3 + momentum * params["dW3_prev"]
    db3_update = -learning_rate * dL_db3 + momentum * params["db3_prev"]

    # Update parameters
    new_params = params.copy()
    new_params["W1"] += dW1_update
    new_params["b1"] += db1_update
    new_params["W2"] += dW2_update
    new_params["b2"] += db2_update
    new_params["W3"] += dW3_update
    new_params["b3"] += db3_update

    # Update momentum terms
    new_params["dW1_prev"] = dW1_update
    new_params["db1_prev"] = db1_update
    new_params["dW2_prev"] = dW2_update
    new_params["db2_prev"] = db2_update
    new_params["dW3_prev"] = dW3_update
    new_params["db3_prev"] = db3_update

    return new_params


# Test with a single sample
X = generate_sample("b", 0.0)
print("Input pattern:")
print_sample(X)
y_target = [1, 0, 0]

params = initialize_parameters()
y_pred = feedforward(X, params)[-1]
loss = calculate_loss(y_target, y_pred)
print(f"1. Objetivo: {y_target}. Predicción: {y_pred}. Loss: {loss}")

params = train_step(X, y_target, params, learning_rate=0.1, momentum=0.1)
y_pred = feedforward(X, params)[-1]
loss = calculate_loss(y_target, y_pred)
print(f"2. Objetivo: {y_target}. Predicción: {y_pred}. Loss: {loss}")

params = train_step(X, y_target, params, learning_rate=0.1, momentum=0.1)
y_pred = feedforward(X, params)[-1]
loss = calculate_loss(y_target, y_pred)
print(f"3. Objetivo: {y_target}. Predicción: {y_pred}. Loss: {loss}")

params = train_step(X, y_target, params, learning_rate=0.1, momentum=0.1)
y_pred = feedforward(X, params)[-1]
loss = calculate_loss(y_target, y_pred)
print(f"4. Objetivo: {y_target}. Predicción: {y_pred}. Loss: {loss}")

Input pattern:
0 0 0 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0 0 0
0 0 1 1 1 1 1 0 0 0
0 0 1 0 0 0 0 1 0 0
0 0 1 0 0 0 0 1 0 0
0 0 1 0 0 0 0 1 0 0
0 0 1 1 1 1 1 0 0 0
0 0 0 0 0 0 0 0 0 0
Parameters initialized:
  W1 shape: (10, 100), b1 shape: (10,)
  W2 shape: (5, 10), b2 shape: (5,)
  W3 shape: (3, 5), b3 shape: (3,)
  Total parameters: 1083
1. Objetivo: [1, 0, 0]. Predicción: [ 0.16544066  0.03604878 -0.40975277]. Loss: 0.28856204783418027
2. Objetivo: [1, 0, 0]. Predicción: [ 0.49042361  0.01223818 -0.26152755]. Loss: 0.10940484309405736
3. Objetivo: [1, 0, 0]. Predicción: [ 7.24420643e-01 -3.75384281e-05 -1.53169147e-01]. Loss: 0.03313492362284643
4. Objetivo: [1, 0, 0]. Predicción: [ 0.85893055 -0.00357482 -0.08731056]. Loss: 0.009178834356804262
